[View in Colaboratory](https://colab.research.google.com/github/battlerhythm/tensorflow/blob/master/5.2%20TensorBoard2.ipynb)

In [1]:
from google.colab import files
files.upload()

Saving data.csv to data.csv


{'data.csv': b'0, 0, 1, 0, 0\n1, 0, 0, 1, 0\n1, 1, 0, 0, 1\n0, 0, 1, 0, 0\n0, 0, 1, 0, 0\n0, 1, 0, 0, 1\n'}

In [0]:
import tensorflow as tf
import numpy as np

data = np.loadtxt('./data.csv', delimiter=',', unpack=True, dtype='float32')

x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

In [0]:
global_step = tf.Variable(0, trainable=False, name='gloabal_step')

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

# with tf.name_scope로 묶은 블록은 텐서보드에서 한 계층 내부를 표현해줍니다.
# 변수 뒤에 name='W1'처럼 이름을 붙이면 텐서보드에서 해당 이름의 변수가 어디에서 사용되는지 쉽게 확인할 수 있습니다.
# 이름은 변수뿐만 아니라 플레이스 홀더, 각각의 연산, 활성화 함수 등 모든 텐서에 붙일 수 있습니다.
with tf.name_scope('layer1'):
    W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.), name='W1')
    L1 = tf.nn.relu(tf.matmul(X, W1))
    
    # TensorBoard Histogram Dashboard
    # https://www.tensorflow.org/programmers_guide/tensorboard_histograms
    tf.summary.histogram("X", X)
    tf.summary.histogram("Weights", W1)
    
with tf.name_scope('layer2'):
    W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.), name='W2')
    L2 = tf.nn.relu(tf.matmul(L1, W2))
    
    tf.summary.histogram("Weights", W2)
    
with tf.name_scope('layer3'):
    W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.), name='W3')
    model = tf.matmul(L2, W3) #출력단에서는 보통 Activation Function을 사용하지 않습니다.
    
    tf.summary.histogram("Weights", W3)
    tf.summary.histogram("Model", model)
    
with tf.name_scope('optimizer'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    train_op = optimizer.minimize(cost, global_step=global_step)
    
    # 손실값을 추적하기 위해 수집할 값을 지정하는 코드를 작성합니다.
    tf.summary.scalar('cost', cost)

In [7]:
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

ckpt = tf.train.get_checkpoint_state('./model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())

# tf.summary.merge_all 함수로 앞서 지정한 텐서들을 수집한 다음
# tf.summary.FileWriter함수를 이용해 그래프와 텐서들의 값을 저장할 디렉터리를 설정합니다.
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('./logs', sess.graph)

for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    print('Step: %d, ' % sess.run(global_step), 'Cost: %.3f' % sess.run(cost, feed_dict={X: x_data, Y: y_data}))
    
    # session.run을 이용해 앞서 merged로 모아둔 텐서의 값들을 계산하여 수집한뒤,
    # writer.add_summary함수를 이용해 해당 값들을 앞서 지정한 디렉토리에 저장합니다.
    summary = sess.run(merged, feed_dict={X: x_data, Y: y_data})
    writer.add_summary(summary, global_step=sess.run(global_step))
    
saver.save(sess, './model/dnn.ckpt', global_step=global_step)

prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

sess.close()

# Consol에서
# tensorboard --logdir=./logs
# output에 출력된 portnumber를 확인하고, 웹브라우저에서 열면 TensorBoard가 보입니다.

INFO:tensorflow:Restoring parameters from ./model/dnn.ckpt-300
Step: 301,  Cost: 0.549
Step: 302,  Cost: 0.549
Step: 303,  Cost: 0.549
Step: 304,  Cost: 0.549
Step: 305,  Cost: 0.549
Step: 306,  Cost: 0.549
Step: 307,  Cost: 0.549
Step: 308,  Cost: 0.549
Step: 309,  Cost: 0.549
Step: 310,  Cost: 0.549
Step: 311,  Cost: 0.549
Step: 312,  Cost: 0.549
Step: 313,  Cost: 0.549
Step: 314,  Cost: 0.549
Step: 315,  Cost: 0.549
Step: 316,  Cost: 0.549
Step: 317,  Cost: 0.549
Step: 318,  Cost: 0.549
Step: 319,  Cost: 0.549
Step: 320,  Cost: 0.549
Step: 321,  Cost: 0.549
Step: 322,  Cost: 0.549
Step: 323,  Cost: 0.549
Step: 324,  Cost: 0.549
Step: 325,  Cost: 0.549
Step: 326,  Cost: 0.549
Step: 327,  Cost: 0.549
Step: 328,  Cost: 0.549
Step: 329,  Cost: 0.549
Step: 330,  Cost: 0.549
Step: 331,  Cost: 0.549
Step: 332,  Cost: 0.549
Step: 333,  Cost: 0.549
Step: 334,  Cost: 0.549
Step: 335,  Cost: 0.549
Step: 336,  Cost: 0.549
Step: 337,  Cost: 0.549
Step: 338,  Cost: 0.549
Step: 339,  Cost: 0.549
S

In [8]:
LOG_DIR = './logs'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -qq
!unzip ngrok-stable-linux-amd64.zip

get_ipython().system_raw('./ngrok http 6006 &')

!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
http://6afbd2b0.ngrok.io
